In [1]:
import FailtogetConv_error as fe
fe.fail_to_get_conv()

1 Physical GPUs, 1 Logical GPUs


In [2]:
import os
from glob import glob
from datetime import datetime

import numpy as np
import tensorflow as tf
from tensorflow.keras import datasets
from tensorflow.keras import layers
import matplotlib.pyplot as plt

%matplotlib inline
%load_ext tensorboard

In [3]:
num_epochs = 10
batch_size = 32

learning_rate = 0.001

dropout_rate = 0.5

input_shape = (32, 32, 1)
num_classes = 10

## build Model

In [4]:
inputs = layers.Input(input_shape)
net = layers.Conv2D(32, (3, 3), padding='SAME')(inputs)
net = layers.Activation('relu')(net)
net = layers.Conv2D(32, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Flatten()(net)
net = layers.Dense(512)(net)
net = layers.Activation('relu')(net)
net = layers.Dropout(dropout_rate)(net)
net = layers.Dense(num_classes)(net)
net = layers.Activation('softmax')(net)

model = tf.keras.Model(inputs=inputs, outputs=net, name='Basic_CNN')

In [5]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

## data preprocess

In [6]:
os.listdir('../dataset/cifar/')

['labels.txt', 'test', 'train']

In [7]:
train_paths = glob('../dataset/cifar/train/*.png')[:1000]
test_paths = glob('../dataset/cifar/test/*.png')[:1000]

len(train_paths), len(test_paths)

(1000, 1000)

In [8]:
def get_class_name(path):
    return path.split('_')[-1].replace('.png', '')

In [9]:
train_labels = [get_class_name(path) for path in train_paths]
class_names = np.unique(train_labels)

In [10]:
def get_label(path):
    fname = tf.strings.split(path, '_')[-1]
    cls_name = tf.strings.regex_replace(fname, '.png', '')
    onehot = tf.cast(cls_name == class_names, tf.uint8)
    return tf.argmax(onehot)

In [11]:
def load_image_label(path):
    # read image
    gfile = tf.io.read_file(path)
    image = tf.io.decode_image(gfile)
    
    # get label
    label =  tf.cast(image, tf.float32) / 255.
    label = get_label(path)
    
    return image, label

In [12]:
def image_preprocess(image, label):
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_flip_left_right(image)
    # image = tf.image.rgb_to_grayscale(image)
    return image, label

In [13]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [14]:
train_dataset = tf.data.Dataset.from_tensor_slices(train_paths)
train_dataset = train_dataset.map(load_image_label, num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.map(image_preprocess, num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.batch(batch_size)
train_dataset = train_dataset.shuffle(buffer_size=len(train_paths))
train_dataset = train_dataset.repeat()

In [15]:
test_dataset = tf.data.Dataset.from_tensor_slices(test_paths)
test_dataset = test_dataset.map(load_image_label, num_parallel_calls=AUTOTUNE)
test_dataset = test_dataset.batch(batch_size)
test_dataset = test_dataset.repeat()

## Callbacks

In [16]:
datetime.now().strftime('%Y%m%d-%H%M%S')

'20210127-163915'

In [17]:
logdir = os.path.join('logs', datetime.now().strftime('%Y%m%d-%H%M%S'))

In [18]:
tensorboard = tf.keras.callbacks.TensorBoard(
    log_dir=logdir,
    write_graph=True,
    write_images=True,
    histogram_freq=1
)

In [19]:
%tensorboard --logdir logs --port 2225

## training

In [20]:
steps_per_epoch = len(train_paths) // batch_size
validation_steps = len(test_paths) // batch_size

model.fit_generator(train_dataset,
                    steps_per_epoch=steps_per_epoch,
                    validation_data=test_dataset,
                    validation_steps=validation_steps,
                    epochs=num_epochs,
                    callbacks=[tensorboard])

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/10
 1/31 [..............................] - ETA: 0s - loss: 52.7616 - accuracy: 0.1875WARNING:tensorflow:From /home/jovyan/.venv/tf2.3.0-keras2.4.0-py3.7-cuda10.1/lib/python3.7/site-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
23/31 [=====================>........] - ETA: 0s - loss: 16.4295 - accuracy: 0.1081

NotFoundError:  No algorithm worked!
	 [[node Basic_CNN/conv2d/Conv2D (defined at <ipython-input-20-a96bac30de81>:9) ]] [Op:__inference_test_function_1773]

Function call stack:
test_function
